<br>
サイドメニューのFile BrowserからTwitterのアーカイブ（ZIPファイル）を「アップロード」（実際にはローカルコピーであり、外部ネットワークへの送信ではありません）した後、上にある早送りボタン（「Restart Kernel and Run All Cells...」）を押してすべてのセルを実行してください。しばらく待つと、この直下のセルの下に入力欄が表示されますので、アップロードしたファイル名を入力してください。アップロード先はrun.ipynbと同じフォルダを想定しています。

In [ ]:
import os

for _ in range(5):
    file_name = input()
    if os.path.isfile(file_name):
        break
    print("入力されたファイル名のファイルが存在しません。正しいファイル名を入力してください。")
    # コピペミスの可能性の通知
    if '.' not in file_name:
        print()
        print("もしかしたら拡張子が抜けているせいかもしれません。確認してみてください。")
else:
    print()
    print("入力されたファイル名のファイルが存在しませんでした。ファイル名を確認した上でこのセルを再実行し、正しいファイル名を入力してください。")

この上に表示される入力欄にアップロードしたファイル名を入力してください。ファイル名が正しく入力されていれば分析結果がページ最下部に表示されます。

In [ ]:
import os
import shutil

unzipped_directory_name = os.path.splitext(file_name)[0]
shutil.unpack_archive(file_name, unzipped_directory_name)
shutil.copy(os.path.join(f'{unzipped_directory_name}{os.sep}data{os.sep}tweets.js'), '.')

In [ ]:
from pathlib import Path
 
read_path = Path('tweets.js')
write_path = Path('tweets.py')
 
# 読み込み
content = read_path.read_text(encoding='utf-8')

# 置換
content = content.replace('window.YTD.tweets.part0', 'value', 1)
content = content.replace('true', 'True')
content = content.replace('false', 'False')

# 書き出し
write_path.write_text(content, encoding='utf-8')

from tweets import value

print(len(value))